In [0]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import re

In [0]:
catalog = "ademianczuk"
database = "ncr"
table = f"{catalog}.{database}.csv_copy_progress"
src_dir = f"/Volumes/{catalog}/{database}/data/csv"
dst_dir = f"/Volumes/{catalog}/{database}/data/loader"

dbutils.fs.mkdirs(dst_dir)

# # Create table once
# spark.sql(f"""
# CREATE TABLE IF NOT EXISTS {table} (
#   filename STRING,
#   seq INT,
#   status STRING,
#   copied_at TIMESTAMP
# ) USING DELTA
# """)

# def is_month_csv(name):
#     return name.startswith("month_") and name.endswith(".csv")

# def month_key(name):
#     m = re.search(r"month[_-](\d{2})\.csv$", name)
#     return int(m.group(1)) if m else 9999

# src_files = [f for f in dbutils.fs.ls(src_dir) if is_month_csv(f.name)]
# src_files = sorted(src_files, key=lambda f: month_key(f.name))

# if not src_files:
#     raise RuntimeError(f"No month_*.csv files found in {src_dir}")

# # Compute next seq by counting distinct filenames already logged as "done"
# done = spark.table(table).filter(col("status") == "done").select("filename").distinct().collect()
# done_set = {r.filename for r in done}

# # Determine the next file (first in order not yet “done”)
# pending = [f for f in src_files if f.name not in done_set]

# if not pending:
#     print(f"Nothing to do. All {len(src_files)} files already copied.")
# else:
#     src = pending[0]
#     dst = f"{dst_dir}/{src.name}"

#     # Idempotent copy
#     exists = False
#     try:
#         _ = dbutils.fs.ls(dst)
#         exists = True
#     except Exception:
#         exists = False

#     if not exists:
#         dbutils.fs.cp(src.path, dst)
#         status = "done"
#         print(f"Copied: {src.path}  ->  {dst}")
#     else:
#         status = "already_present"
#         print(f"Already present, logging and skipping: {dst}")

#     # Append a log row
#     seq_num = 1 + spark.table(table).count()
#     spark.createDataFrame(
#         [Row(filename=src.name, seq=seq_num, status=status)]
#     ).withColumn("copied_at", expr("current_timestamp()")) \
#      .write.mode("append").saveAsTable(table)

#     remaining = len(pending) - 1
#     print(f"Progress: {len(done_set) + 1}/{len(src_files)} copied. {remaining} remaining.")